# Initialize

Mount Google Drive

**Raw Data Source:**
https://github.com/GeoDS/COVID19USFlows#dataset-structure

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
!pip install haversine

In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import haversine as hs
from matplotlib import pyplot as plt
from pathlib import Path

Define Functions

In [8]:
def hav(x, y):
    return hs.haversine(x, y)

In [9]:
def w_avg(df, values, weights):
    d = df[values]
    w = df[weights]
    return (d * w).sum() / w.sum()

In [87]:
#set origin vs destination weighting
origin_weight = 0.5
dest_weight = (1 - origin_weight)

# Reingest Condensed CSVs

In [69]:
#Reference normalized origin and destination files
mobility_raw_origin_path = 'drive/MyDrive/Education/UVA_MSDS/Capstone/MobilityData/mobility_01_01_2019_to_04_15_2021_origin_raw.csv'
mobility_raw_dest_path = 'drive/MyDrive/Education/UVA_MSDS/Capstone/MobilityData/mobility_01_01_2019_to_04_15_2021_dest_raw.csv'

In [70]:
#Read in normalized origin and destination files
mobility_raw_origin = pd.read_csv(mobility_raw_origin_path)
mobility_raw_dest = pd.read_csv(mobility_raw_dest_path)

In [82]:
mobility_raw_origin

mobility_index_origin  mobility_index_norm_origin
dates      FIPS                                                    
2021-01-01 1001               34.878216                    0.017446
           1003               39.222893                    0.019626
           1005               42.017411                    0.021028
           1007               41.598706                    0.020818
           1009               43.600470                    0.021822
...                                 ...                         ...
2019-12-31 56037              56.665313                    0.028377
           56039              63.989467                    0.032051
           56041              50.927906                    0.025498
           56043              50.712056                    0.025390
           56045              51.263824                    0.025667

[2625786 rows x 2 columns]

In [83]:
mobility_raw_dest

mobility_index_dest  mobility_index_norm_dest
dates      FIPS                                                
2021-01-01 1001             36.672983                  0.018418
           1003             91.804502                  0.054620
           1005             42.414662                  0.022188
           1007             32.564355                  0.015720
           1009             34.131222                  0.016749
...                               ...                       ...
2019-12-31 56037            33.445273                  0.016299
           56039           105.745500                  0.063775
           56041            33.148023                  0.016103
           56043            26.471431                  0.011719
           56045            23.592456                  0.009829

[2626206 rows x 2 columns]

In [75]:
mobility_raw_origin.rename(columns = {'mobility_index':'mobility_index_origin', 'mobility_index_norm':'mobility_index_norm_origin'}, inplace = True)

In [76]:
mobility_raw_origin.set_index(['dates','FIPS'], inplace = True)

In [77]:
mobility_raw_dest.rename(columns = {'mobility_index':'mobility_index_dest', 'mobility_index_norm':'mobility_index_norm_dest'}, inplace = True)

In [78]:
mobility_raw_dest.set_index(['dates','FIPS'], inplace = True)

In [85]:
mobility_raw_full = pd.concat([mobility_raw_origin, mobility_raw_dest], axis=1)

In [94]:
mobility_raw_full

mobility_index_full  mobility_index_norm_full
dates      FIPS                                                
2019-01-01 1001             33.175754                  0.015959
           1003             56.165482                  0.030957
           1005             34.466613                  0.016760
           1007             34.320899                  0.016553
           1009             41.794873                  0.020787
...                               ...                       ...
2021-04-15 56037            49.235512                  0.024892
           56039            64.770538                  0.033744
           56041            34.819218                  0.017079
           56043            51.443741                  0.028141
           56045            31.924263                  0.015232

[2626206 rows x 2 columns]

In [88]:
mobility_raw_full['mobility_index_full'] = (mobility_raw_full.mobility_index_origin * origin_weight) + (mobility_raw_full.mobility_index_dest * dest_weight)

In [90]:
mobility_raw_full['mobility_index_norm_full'] = (mobility_raw_full.mobility_index_norm_origin * origin_weight) + (mobility_raw_full.mobility_index_norm_dest * dest_weight)

In [93]:
mobility_raw_full.drop(columns=['mobility_index_origin', 'mobility_index_norm_origin','mobility_index_dest','mobility_index_norm_dest'], inplace = True)

In [95]:
mobility_raw_full.to_csv('drive/MyDrive/Education/UVA_MSDS/Capstone/MobilityData/mobility_01_01_2019_to_04_15_2021_full_raw.csv', index= False)

In [96]:
mobility_full = mobility_raw_full.pivot_table(index = 'FIPS', columns = 'dates', values = 'mobility_index_norm_full')

In [97]:
mobility_full

dates,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,2019-01-10,...,2021-04-06,2021-04-07,2021-04-08,2021-04-09,2021-04-10,2021-04-11,2021-04-12,2021-04-13,2021-04-14,2021-04-15
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,0.015959,0.016867,0.016126,0.017887,0.015713,0.014664,0.015912,0.015505,0.016656,0.016629,...,0.026281,0.021832,0.021777,0.026675,0.025544,0.020984,0.022734,0.026209,0.026953,0.023470
1003,0.030957,0.028808,0.026763,0.028414,0.028975,0.022882,0.019184,0.019997,0.020760,0.020735,...,0.088409,0.078318,0.081630,0.079604,0.060590,0.048981,0.036425,0.034527,0.034080,0.034536
1005,0.016760,0.018732,0.018648,0.022585,0.022034,0.018123,0.014451,0.016224,0.015590,0.016737,...,0.026503,0.026289,0.026602,0.032225,0.032763,0.023216,0.020984,0.025113,0.020619,0.018883
1007,0.016553,0.017460,0.017937,0.020505,0.019244,0.017302,0.014680,0.016198,0.017988,0.017290,...,0.032646,0.029290,0.033894,0.030915,0.026368,0.025641,0.026766,0.025413,0.028834,0.027051
1009,0.020787,0.020696,0.021515,0.023183,0.025022,0.020061,0.015432,0.017291,0.017241,0.018002,...,0.028837,0.025848,0.024233,0.024053,0.024133,0.020961,0.021485,0.026974,0.027106,0.023897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,0.015641,0.014939,0.015805,0.019266,0.019901,0.016037,0.014742,0.015675,0.015498,0.016840,...,0.027039,0.028211,0.025908,0.029268,0.027540,0.024261,0.023140,0.022272,0.027403,0.024892
56039,0.034247,0.035444,0.032044,0.035602,0.032211,0.023194,0.022202,0.024234,0.023634,0.022147,...,0.049816,0.043703,0.044209,0.039561,0.042331,0.032518,0.039082,0.040493,0.037034,0.033744
56041,0.015074,0.015730,0.013908,0.014464,0.015788,0.012496,0.011584,0.012663,0.013913,0.013123,...,0.029418,0.028119,0.024731,0.024166,0.021904,0.021173,0.020057,0.019438,0.017186,0.017079


In [98]:
mobility_full.to_csv('drive/MyDrive/Education/UVA_MSDS/Capstone/MobilityData/mobility_01_01_2019_to_04_15_2021_full.csv', index = False)